In [133]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [220]:
df = pd.read_excel('D:\\wallet anomaly\\User Ballance Claim Approved_1.xlsx')

In [221]:
df.to_csv('D:\\wallet anomaly\\User Ballance Claim Approved_1.csv')

In [222]:
df.head()

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType
0,ceb04f25-28c1-4f39-a6c4-e3104158b0f8,2023-10-24 15:07:07.491515,2023-10-24 15:13:30.060336,49d09202-e746-4ead-834f-973eaa08cf85,18.0,1,2
1,13053d68-dfd4-4b7f-9316-62012e3403b5,2023-10-25 08:24:39.675849,2023-10-25 08:31:26.172195,5f388a02-3dce-42b0-8857-7adf3cda85a2,84.0,1,2
2,2bb9467c-09da-43b1-92df-676dd8eab993,2023-10-24 15:42:30.788446,2023-10-24 15:52:30.089504,d8f70ebb-e38f-466f-929e-5c18fdd92b66,191.0,1,2
3,c8d66bc8-eed9-4c46-89b4-77a319de3243,2023-10-25 08:25:47.120024,2023-10-25 08:31:42.394063,6cd38d23-21b6-44bf-94f8-3e78ac10b457,69.0,1,2
4,b2ece785-8270-4929-93c6-73f50a67a424,2023-10-24 16:06:19.047844,2023-10-24 16:17:04.426334,6ab3ae8c-ae16-4b21-8ed5-7070b7630e2c,2.0,1,2


In [223]:
#using only courier's data
data = df[df['UserType'] == 2]

In [224]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 170481 entries, 0 to 171485
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Id         170481 non-null  object 
 1   CreatedAt  170481 non-null  object 
 2   UpdatedAt  170481 non-null  object 
 3   UserId     170481 non-null  object 
 4   Amount     170481 non-null  float64
 5   ClaimType  170481 non-null  int64  
 6   UserType   170481 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 10.4+ MB


In [225]:
data.columns

Index(['Id', 'CreatedAt', 'UpdatedAt', 'UserId', 'Amount', 'ClaimType',
       'UserType'],
      dtype='object')

In [226]:
# # Convert the array to a DataFrame
# df = pd.DataFrame(df['UserId'].unique(), columns=['uniqueid'])

# # Specify the Excel file path
# excel_file_path = 'D:\\wallet anomaly\\uniqueid_excel_file.xlsx'

# # Save the DataFrame to an Excel file
# df.to_excel(excel_file_path, index=False)

# print(f"Excel file '{excel_file_path}' has been created.")

In [227]:
data.duplicated().sum()

0

In [228]:
data['CreatedAt'] = pd.to_datetime(data['CreatedAt'])
#data['Updated at'] = pd.to_datetime(data['Updated at'])

C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\1066382839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CreatedAt'] = pd.to_datetime(data['CreatedAt'])


In [229]:
data['UserType'].value_counts()

UserType
2    170481
Name: count, dtype: int64

In [230]:
X = data[['CreatedAt', 'Amount','UserType']]

In [231]:
X.head()

,CreatedAt,Amount,UserType
0,2023-10-24 15:07:07.491515,18.0,2
1,2023-10-25 08:24:39.675849,84.0,2
2,2023-10-24 15:42:30.788446,191.0,2
3,2023-10-25 08:25:47.120024,69.0,2
4,2023-10-24 16:06:19.047844,2.0,2


In [232]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 170481 entries, 0 to 171485
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   CreatedAt  170481 non-null  datetime64[ns]
 1   Amount     170481 non-null  float64       
 2   UserType   170481 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 5.2 MB


In [233]:
X = data.select_dtypes(include=['number', 'bool'])
isolation_forest_model = IsolationForest(contamination=0.01)
isolation_forest_model.fit(X)

IsolationForest(contamination=0.01)

In [234]:
predictions = isolation_forest_model.predict(X)

In [235]:
data['predictions'] = predictions

C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\595567529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['predictions'] = predictions


In [236]:
data[data['predictions'] == -1].head()

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions
64,cef48f50-923b-4745-8b5c-b49f798d058a,2023-10-25 01:43:20.032213,2023-10-25 05:02:00.204694,d87a7dfe-dc55-41e6-89cb-5baaf47f02b3,999.0,1,2,-1
528,43df28bd-f343-460f-afb1-1621be5950b8,2023-10-27 00:58:57.893621,2023-10-27 01:01:23.984908,d87a7dfe-dc55-41e6-89cb-5baaf47f02b3,901.0,1,2,-1
544,296fbae6-e557-4b25-8f5d-a2c53181682c,2023-10-27 10:03:57.649307,2023-10-27 10:14:15.594813,185614f7-8fed-48f1-b9fe-73714a8f3c4f,966.0,1,2,-1
904,c46978c4-a704-4f19-b3e8-a53cdc0416da,2023-10-27 20:12:24.710607,2023-10-27 20:36:51.297628,ef9f0e46-79d3-4851-a910-9a467b25f136,904.0,1,2,-1
1024,07bff92a-2922-4df0-858f-8df9d6b882d4,2023-10-29 11:38:31.647568,2023-10-29 11:41:11.721755,f2516bd5-c6bb-4d2a-ad0b-22114ff51618,849.0,1,2,-1


In [237]:
data[data['predictions'] == -1].shape

(1233, 8)

In [238]:
# Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [239]:
# Reduce dimensionality with PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

In [240]:
# Build Autoencoder model
model = Sequential()
model.add(Dense(3, input_dim=3, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(3, activation='linear'))  # Update to match the number of components in PCA
model.add(Dense(3, activation='linear'))

model.compile(optimizer='adam', loss='mse')
model.fit(X_pca, X_pca, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50
4795/4795 [==============================] - 7s 1ms/step - loss: 0.0424 - val_loss: 3.2856e-05
Epoch 2/50
4795/4795 [==============================] - 6s 1ms/step - loss: 0.0026 - val_loss: 6.6915e-10
Epoch 3/50
4795/4795 [==============================] - 6s 1ms/step - loss: 2.7526e-07 - val_loss: 8.5894e-05
Epoch 4/50
4795/4795 [==============================] - 6s 1ms/step - loss: 6.3962e-07 - val_loss: 7.3558e-13
Epoch 5/50
4795/4795 [==============================] - 6s 1ms/step - loss: 4.9421e-06 - val_loss: 9.5049e-12
Epoch 6/50
4795/4795 [==============================] - 7s 1ms/step - loss: 1.2985e-05 - val_loss: 3.3669e-10
Epoch 7/50
4795/4795 [==============================] - 7s 1ms/step - loss: 1.3248e-05 - val_loss: 9.0585e-08
Epoch 8/50
4795/4795 [==============================] - 6s 1ms/step - loss: 1.7106e-06 - val_loss: 3.9860e-09
Epoch 9/50
4795/4795 [==============================] - 6s 1ms/step - loss: 1.7392e-05 - val_loss: 6.8765e-09
Epoch 10/50
4795/4

In [241]:
# Predict reconstruction errors
X_pred = model.predict(X_pca)
mse = np.mean(np.power(X_pca - X_pred, 2), axis=1)
data['is_outlier_autoencoder'] = mse > np.percentile(mse, 99)

5328/5328 [==============================] - 5s 959us/step


C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\529401454.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['is_outlier_autoencoder'] = mse > np.percentile(mse, 99)


In [242]:
data[data['is_outlier_autoencoder'] == True].head()

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder
4,b2ece785-8270-4929-93c6-73f50a67a424,2023-10-24 16:06:19.047844,2023-10-24 16:17:04.426334,6ab3ae8c-ae16-4b21-8ed5-7070b7630e2c,2.0,1,2,1,True
145,93b2350f-8d87-40b4-aeb5-dd94d7b648c1,2023-10-25 09:23:01.348156,2023-10-25 09:31:56.525448,f2516bd5-c6bb-4d2a-ad0b-22114ff51618,2.0,1,2,1,True
232,7d16972c-ca72-4726-8a8b-e0a4ebecef4a,2023-10-25 16:06:27.610615,2023-10-25 16:16:59.543575,a0ee141b-d713-4faf-9662-bb734c3a9063,2.0,1,2,1,True
425,892ad23c-c1b2-4ac6-870a-f82b3c5c2de0,2023-10-26 15:30:14.304787,2023-10-26 15:46:07.569556,bd471329-ec37-4258-8c57-3551dcf05ae8,2.0,1,2,1,True
512,3f12cd5b-3086-42d2-ba33-9d159cdba7ee,2023-10-27 07:49:14.904629,2023-10-27 08:00:36.089218,380ad6d4-80f5-48e9-9440-afaf4300393b,2.0,1,2,1,True


In [243]:
data[data['is_outlier_autoencoder'] == True].shape

(1681, 9)

In [244]:
data[data['is_outlier_autoencoder'] == True].loc[data['predictions'] == -1].head()

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder


In [245]:
data[data['is_outlier_autoencoder'] == True].loc[data['predictions'] == -1].shape

(0, 9)

In [246]:
#using dbscan model

# Split data into chunks
chunk_size = 1000
num_chunks = len(X) // chunk_size + 1

# Initialize result column
data['is_outlier_dbscan'] = False

# Train DBSCAN model iteratively
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size

    # Adjust the parameters as needed
    dbscan_model = DBSCAN(eps=3, min_samples=5)
    
    # Fit and predict on the current chunk
    chunk_outliers = dbscan_model.fit_predict(X_scaled[start_idx:end_idx]) == -1
    
    # Update the result column for the current chunk
    data['is_outlier_dbscan'].iloc[start_idx:end_idx] = chunk_outliers

C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\388879235.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['is_outlier_dbscan'] = False
C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\388879235.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['is_outlier_dbscan'].iloc[start_idx:end_idx] = chunk_outliers
C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\388879235.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [247]:
data[data['is_outlier_dbscan'] == True]

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder,is_outlier_dbscan
46464,b3d7f306-e1ac-4e1e-9275-7b9689c6559a,2023-11-08 22:05:24.957507,2023-11-08 22:07:13.538573,b957927a-150f-46c4-bd14-ac617c2558f3,3950.0,1,2,1,False,True
46528,8c69a559-f485-4e3e-a146-da61b515c893,2023-11-08 22:18:47.564054,2023-11-08 22:19:29.503397,b957927a-150f-46c4-bd14-ac617c2558f3,10000.0,1,2,1,False,True
46725,2c36cadc-d0d8-4ff4-bf25-037b7492710c,2023-11-08 22:23:45.268898,2023-11-08 22:23:58.331033,b957927a-150f-46c4-bd14-ac617c2558f3,20000.0,1,2,1,False,True
46868,24a7c1f6-2483-4ef3-9523-e97664de62bc,2023-11-08 22:29:48.498829,2023-11-08 22:30:07.087649,b957927a-150f-46c4-bd14-ac617c2558f3,20000.0,1,2,1,False,True
47275,54ce68cd-75d5-4a9a-9fdb-1b5a141f529e,2023-11-08 23:18:24.652364,2023-11-08 23:19:09.450788,b957927a-150f-46c4-bd14-ac617c2558f3,9300.0,1,2,1,False,True


In [248]:
data[data['is_outlier_dbscan'] == True].shape

(5, 10)

In [250]:
data[(data['predictions'] == -1) & (data['is_outlier_dbscan'] == True)]

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder,is_outlier_dbscan


In [249]:
 data[(data['predictions'] == -1) & (data['is_outlier_autoencoder'] == True) & (data['is_outlier_dbscan'] == True)]

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder,is_outlier_dbscan


In [252]:
 data[(data['predictions'] == -1) | (data['is_outlier_autoencoder'] == True) | (data['is_outlier_dbscan'] == True)].shape

(2919, 10)

In [286]:
##### partner anomalies

partner = df[df['UserType'] == 4]

In [287]:
partner.head()

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType
70,0f2a9f70-4930-43e3-8150-f3aca8d47c49,2023-10-24 15:41:17.297502,2023-10-25 06:55:57.065888,b08ad201-dcad-405f-bb9d-757171f9e140,437.0,1,4
71,66de2070-9555-44a7-8698-8d42fe495e77,2023-10-24 15:44:03.957502,2023-10-25 06:57:02.572452,877b7b58-1051-4713-acd7-e9813e65388b,3010.0,1,4
76,65e366fd-a823-41d1-a0bb-bddf0d640752,2023-10-24 18:16:42.487190,2023-10-25 06:58:23.328128,08a91339-d258-44d4-91b6-2ebabff6107d,100.0,1,4
94,1c2d2fa9-15c8-4ebe-8fe2-7c09226da986,2023-10-24 22:05:54.439023,2023-10-25 06:59:38.574906,03fc360c-2262-44c6-b225-8a3bcb7a709e,98.0,1,4
157,608da836-3dcc-497d-93ba-792b80a48dfd,2023-10-25 11:18:12.361334,2023-10-25 11:21:25.595867,d9e2344b-2000-4c52-9a6a-ce36b00ae6a1,10000.0,1,4


In [288]:
partner['CreatedAt'] = pd.to_datetime(partner['CreatedAt'])
#partner['Updated at'] = pd.to_datetime(partner['Updated at'])

C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\1559061742.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner['CreatedAt'] = pd.to_datetime(partner['CreatedAt'])


In [289]:
X = partner[['CreatedAt', 'Amount','UserType']]

In [290]:
X.dtypes

CreatedAt    datetime64[ns]
Amount              float64
UserType              int64
dtype: object

In [292]:
X = partner.select_dtypes(include=['number', 'bool'])
isolation_forest_model = IsolationForest(contamination=0.001)
isolation_forest_model.fit(X)

IsolationForest(contamination=0.001)

In [293]:
predictions = isolation_forest_model.predict(X)

In [294]:
partner['predictions'] = predictions

C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\1212037506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner['predictions'] = predictions


In [309]:
partner[partner['predictions'] == -1]

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder,is_outlier_dbscan
155892,0ea48287-e162-4022-b6c9-889b435ccaaa,2023-10-03 08:41:40.487753,2023-10-03 08:46:58.538033,d9e2344b-2000-4c52-9a6a-ce36b00ae6a1,30000.0,1,4,-1,False,True
159687,5447db45-f4fb-46c5-92b7-ba505f52c120,2023-10-18 05:26:20.571670,2023-10-18 08:05:41.380686,ded3196c-25f8-4372-8aa6-8d5c7d00d1a9,30000.0,1,4,-1,False,True


In [296]:
# Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [297]:
# Reduce dimensionality with PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

In [311]:
# Build Autoencoder model
model = Sequential()
model.add(Dense(3, input_dim=3, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(3, activation='linear'))  # Update to match the number of components in PCA
model.add(Dense(3, activation='linear'))

model.compile(optimizer='adam', loss='mse')
model.fit(X_pca, X_pca, epochs=50, batch_size=32, validation_split=0.01)

Epoch 1/50
32/32 [==============================] - 1s 6ms/step - loss: 0.6126 - val_loss: 0.5854
Epoch 2/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4546 - val_loss: 0.4596
Epoch 3/50
32/32 [==============================] - 0s 2ms/step - loss: 0.3755 - val_loss: 0.3875
Epoch 4/50
32/32 [==============================] - 0s 2ms/step - loss: 0.3263 - val_loss: 0.3472
Epoch 5/50
32/32 [==============================] - 0s 2ms/step - loss: 0.2935 - val_loss: 0.3167
Epoch 6/50
32/32 [==============================] - 0s 2ms/step - loss: 0.2658 - val_loss: 0.2926
Epoch 7/50
32/32 [==============================] - 0s 3ms/step - loss: 0.2431 - val_loss: 0.2695
Epoch 8/50
32/32 [==============================] - 0s 2ms/step - loss: 0.2214 - val_loss: 0.2453
Epoch 9/50
32/32 [==============================] - 0s 2ms/step - loss: 0.1983 - val_loss: 0.2214
Epoch 10/50
32/32 [==============================] - 0s 2ms/step - loss: 0.1753 - val_loss: 0.1953
Epoch 11/50
32/32 [

In [299]:
# Predict reconstruction errors
X_pred = model.predict(X_pca)
mse = np.mean(np.power(X_pca - X_pred, 2), axis=1)
partner['is_outlier_autoencoder'] = mse > np.percentile(mse, 99)

32/32 [==============================] - 0s 930us/step


C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\2396363467.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner['is_outlier_autoencoder'] = mse > np.percentile(mse, 99)


In [308]:
partner[partner['is_outlier_autoencoder'] == True]

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder,is_outlier_dbscan
166,86ff01aa-4aac-496f-a3fe-0b09e65e9a32,2023-10-25 11:43:36.579615,2023-10-25 11:44:42.502785,1842df14-389e-4fad-b754-fc87805c76e4,1000.0,1,4,1,True,False
56806,c916d72e-3600-4590-a1f0-287e5fd0fce4,2023-11-09 11:29:05.880166,2023-11-10 12:13:07.964284,0ae126fe-2836-4503-b881-73abe2d24991,1004.0,1,4,1,True,False
128088,2eeeb427-d82a-49fc-96ae-03d88bdb8408,2023-12-02 11:42:01.186412,2023-12-02 11:51:57.435512,5f3b13fb-a7e8-4c59-b6d4-0b40a6b3462e,1000.0,1,4,1,True,False
136496,67bd03e8-18f7-4fd3-bc12-91b232ca4dfa,2023-12-13 10:54:45.026303,2023-12-13 10:56:37.949036,8945b252-5bfe-4513-b5c6-2c7bfa3bd417,1000.0,1,4,1,True,False
146727,88a0c434-64cf-4764-9021-1282abec33ac,2023-08-21 12:32:21.531168,2023-08-21 12:35:05.343334,59146675-3093-4477-b135-a707924c4b9b,1000.0,1,4,1,True,False
147426,ece70b3a-f824-4103-b7d7-c08dc231a67f,2023-08-25 17:24:24.930994,2023-08-26 06:47:34.058162,b38e2370-db69-4e99-8777-6a01f8b6fd98,1000.0,1,4,1,True,False
150289,66a469ce-39d4-4657-866b-7d02f26bf115,2023-11-19 11:06:13.557636,2023-11-19 11:28:47.350838,cbab7cfa-c232-464b-829f-3dfcb7cdcb37,1000.0,1,4,1,True,False
152714,333b0be1-0ad9-4c65-92f1-1df793684a0b,2023-09-21 10:58:09.827476,2023-09-21 11:00:58.267057,3cd568ab-adf5-4cae-aaf2-b379e4e9ae78,1000.0,1,4,1,True,False
157798,39bc7aba-da55-449b-a50c-ab05db5aace6,2023-10-10 13:04:57.318650,2023-10-11 06:39:18.873657,e3dce6d1-3ced-4c9e-9015-034592707eae,1000.0,1,4,1,True,False
157799,3642fd5e-549f-45c6-b200-24a7813ada15,2023-10-10 12:16:48.439289,2023-10-11 07:26:23.047544,f5a9e37f-f5a4-44d1-87ee-cb990b9bfe6f,1000.0,1,4,1,True,False


In [301]:
partner[partner['is_outlier_autoencoder'] == True].shape

(11, 9)

In [302]:
partner[partner['is_outlier_autoencoder'] == True].loc[partner['predictions'] == -1]

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder


In [303]:
#using dbscan model

# Split data into chunks
chunk_size = 10
num_chunks = len(X) // chunk_size + 1

# Initialize result column
partner['is_outlier_dbscan'] = False

# Train DBSCAN model iteratively
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size

    # Adjust the parameters as needed
    dbscan_model = DBSCAN(eps=3, min_samples=5)
    
    # Fit and predict on the current chunk
    chunk_outliers = dbscan_model.fit_predict(X_scaled[start_idx:end_idx]) == -1
    
    # Update the result column for the current chunk
    partner['is_outlier_dbscan'].iloc[start_idx:end_idx] = chunk_outliers

C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\188792243.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner['is_outlier_dbscan'] = False
C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\188792243.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner['is_outlier_dbscan'].iloc[start_idx:end_idx] = chunk_outliers
C:\Users\shgardi\AppData\Local\Temp\ipykernel_16016\188792243.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [304]:
partner[partner['is_outlier_dbscan'] == True]

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder,is_outlier_dbscan
997,0212de9e-a69e-4920-a848-073a8aecfbcb,2023-10-29 08:43:14.429967,2023-10-29 08:45:47.428783,7fc7a5fd-f4f1-44d6-ae99-a24f9857aa8c,19786.0,1,4,1,False,True
1051,309ef8b3-083a-41ee-9cf8-f84479d545d2,2023-10-29 12:06:35.904213,2023-10-29 12:12:09.209611,9ee008e8-23cb-4cb6-a546-e56b49ebb77b,11469.0,1,4,1,False,True
1601,08abac7c-48ab-4ecf-8c4d-cfe813d99481,2023-10-30 12:16:58.455528,2023-10-30 12:26:11.196342,7fc7a5fd-f4f1-44d6-ae99-a24f9857aa8c,23942.0,1,4,1,False,True
2988,2f2addf0-04c7-4df3-9408-3d33af2cb449,2023-11-05 08:03:49.125094,2023-11-05 08:21:03.399859,b20045b6-3ef3-4ac9-9ec4-ed07a95b25ef,11614.0,1,4,1,False,True
137058,ae0032a9-1b18-4712-b6db-9de62a4e2c2b,2023-12-13 11:31:37.829570,2023-12-13 11:36:29.652610,4c72735a-9da9-41f7-8419-5b72e7360f5a,18190.0,1,4,1,False,True
137382,7c1587c6-4f66-417d-aa36-9eaf7f954324,2023-12-03 12:23:39.599996,2023-12-03 12:38:30.748451,de35b2e0-074d-4f26-9a44-644ed5547fa5,17816.0,1,4,1,False,True
149395,06671560-52d4-48e1-a964-b2089dcdd205,2023-09-04 07:30:09.265623,2023-09-04 07:39:48.298342,6680229d-d9d1-4c1d-9ec1-35a636686ebe,18093.0,1,4,1,False,True
149439,582f54cb-66d5-4fa4-b96d-ef2b3751622f,2023-09-04 10:24:42.041084,2023-09-04 10:26:25.110390,ed397dd7-229e-4212-a91a-26693f756661,15900.0,1,4,1,False,True
149970,1a44411c-2be7-4b21-be28-6053c626a96c,2023-09-07 07:15:10.876630,2023-09-07 07:18:28.920952,482aef6b-4564-479b-a680-fad488f9f69d,15000.0,1,4,1,False,True
152339,a6a188a7-4794-4d3b-8976-6abfb0f212e7,2023-09-19 08:57:04.500199,2023-09-19 09:00:39.914194,a05afe21-fa37-4945-a0a2-4dbba8739826,27407.0,1,4,1,False,True


In [305]:
partner[(partner['predictions'] == -1) | (partner['is_outlier_autoencoder'] == True) | (partner['is_outlier_dbscan'] == True)]

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder,is_outlier_dbscan
166,86ff01aa-4aac-496f-a3fe-0b09e65e9a32,2023-10-25 11:43:36.579615,2023-10-25 11:44:42.502785,1842df14-389e-4fad-b754-fc87805c76e4,1000.0,1,4,1,True,False
997,0212de9e-a69e-4920-a848-073a8aecfbcb,2023-10-29 08:43:14.429967,2023-10-29 08:45:47.428783,7fc7a5fd-f4f1-44d6-ae99-a24f9857aa8c,19786.0,1,4,1,False,True
1051,309ef8b3-083a-41ee-9cf8-f84479d545d2,2023-10-29 12:06:35.904213,2023-10-29 12:12:09.209611,9ee008e8-23cb-4cb6-a546-e56b49ebb77b,11469.0,1,4,1,False,True
1601,08abac7c-48ab-4ecf-8c4d-cfe813d99481,2023-10-30 12:16:58.455528,2023-10-30 12:26:11.196342,7fc7a5fd-f4f1-44d6-ae99-a24f9857aa8c,23942.0,1,4,1,False,True
2988,2f2addf0-04c7-4df3-9408-3d33af2cb449,2023-11-05 08:03:49.125094,2023-11-05 08:21:03.399859,b20045b6-3ef3-4ac9-9ec4-ed07a95b25ef,11614.0,1,4,1,False,True
56806,c916d72e-3600-4590-a1f0-287e5fd0fce4,2023-11-09 11:29:05.880166,2023-11-10 12:13:07.964284,0ae126fe-2836-4503-b881-73abe2d24991,1004.0,1,4,1,True,False
128088,2eeeb427-d82a-49fc-96ae-03d88bdb8408,2023-12-02 11:42:01.186412,2023-12-02 11:51:57.435512,5f3b13fb-a7e8-4c59-b6d4-0b40a6b3462e,1000.0,1,4,1,True,False
136496,67bd03e8-18f7-4fd3-bc12-91b232ca4dfa,2023-12-13 10:54:45.026303,2023-12-13 10:56:37.949036,8945b252-5bfe-4513-b5c6-2c7bfa3bd417,1000.0,1,4,1,True,False
137058,ae0032a9-1b18-4712-b6db-9de62a4e2c2b,2023-12-13 11:31:37.829570,2023-12-13 11:36:29.652610,4c72735a-9da9-41f7-8419-5b72e7360f5a,18190.0,1,4,1,False,True
137382,7c1587c6-4f66-417d-aa36-9eaf7f954324,2023-12-03 12:23:39.599996,2023-12-03 12:38:30.748451,de35b2e0-074d-4f26-9a44-644ed5547fa5,17816.0,1,4,1,False,True


In [306]:
partner_results = partner[(partner['predictions'] == -1) & (partner['is_outlier_autoencoder'] == True) & (partner['is_outlier_dbscan'] == True)]

In [307]:
partner_results

,Id,CreatedAt,UpdatedAt,UserId,Amount,ClaimType,UserType,predictions,is_outlier_autoencoder,is_outlier_dbscan


In [ ]:
partner_results.to_excel('D:\\wallet anomaly\\Partner_results_anomalies_withdrawl.xlsx')

In [ ]:
import sys

print("Python version")
print(sys.version)
print("Version info.")
print(sys.version_info)

Python version
3.8.0 (tags/v3.8.0:fa919fd, Oct 14 2019, 19:37:50) [MSC v.1916 64 bit (AMD64)]
Version info.
sys.version_info(major=3, minor=8, micro=0, releaselevel='final', serial=0)
